# K-Nearest Neighbors
---
K-Nearest Neighbors is a supervised machine learning algorithm that looks at the closest *k*  existing points to a specific data point, then looks at the labels of those points in order to estimate the label of the new point. 

In this notebook, we will be assigning *k* a value of 4, which means we will be looking at the 4 closest points to predict the label of any new data.

In order to determine what the closest points are, we will be using the **Distance Formula** to calculate the distance between two points ($p_1$ and $p_2$) which is: $$d(p_1,p_2)=\sqrt{(x_{p_1}-x_{p_2})^2+(y_{p_1}-y_{p_2})^2}$$

---


In [1]:
# Distance formula function
function distance(p1, p2)
    return sqrt(sum((p1[i]-p2[i])^2 for i in 1:length(p1)))
    end

distance (generic function with 1 method)

---
Once we have the distance function established, we then need to apply it to our data to get the distance from the test point to every other point in our data. After that we can sort the results and return the first *k* results.

---

In [2]:
# KNN Algorithm
function KNN(target, x_array, label_array, k)
    
    # Determine distance between target point and all other points
    distance_array = [(x_array[i], label_array[i], distance(target, x_array[i]))
                        for i = 1:length(x_array) 
                        if target != x_array[i]
                        ]
    
    # Find and return 'k' closest points
    sort!(distance_array, by = x -> x[3])
    return distance_array[1:k]
end 

KNN (generic function with 1 method)

---
With the K-Nearest neighbors algorithm now complete, it's time to apply it to a dataset! Let's work with the Iris dataset from the RDatasets package. First thing we need to do is read in the data, then we can subdivide it into *training data* and *test data.* The training data is what we will use within our K-Nearest Neighbors algorithm to predict the label of the test points, which will come from the data set aside for testing. For this exercise we will try to predict the species of iris using **SepalLength, SepalWidth,** and **PetalLength.**

---

In [3]:
# Read in data and split 20% of it off for testing (10 obs/label)
using RDatasets
using Plots
using CSV

iris = dataset("datasets","iris")

# Create array of data for variables of interest
training_data = [x for x in zip(iris.SepalLength[1:40], iris.SepalWidth[1:40], iris.PetalLength[1:40])]
append!(training_data, (iris.SepalLength[i], iris.SepalWidth[i], iris.PetalLength[i]) for i = 51:90)
append!(training_data, (iris.SepalLength[i], iris.SepalWidth[i], iris.PetalLength[i]) for i = 101:140)

# Create array of test data
test_data = [x for x in zip(iris.SepalLength[41:50], iris.SepalWidth[41:50], iris.PetalLength[41:50])]
append!(test_data, (iris.SepalLength[i], iris.SepalWidth[i], iris.PetalLength[i]) for i = 91:100)
append!(test_data, (iris.SepalLength[i], iris.SepalWidth[i], iris.PetalLength[i]) for i = 141:150)

# Create array of labels
label_data = [iris.Species[i] for i = 1:40]
append!(label_data, iris.Species[i] for i = 51:90)
append!(label_data, iris.Species[i] for i = 101:140)

120-element CategoricalArray{String,1,UInt8}:
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 ⋮
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

---
Since we know the labels of every point in our test set, it's very easy to test the accuracy of our K-Nearest Neighbors algorithm. With the knowledge that entries 21 through 30 are all *virginica*, let's use entry 26 to test our algorithm and see what label it predicts for the test point.

---

In [27]:
# Testing a point
println("KNN Algorithm Results:")
KNN_result = KNN(test_data[26], training_data, label_data, 4)
for i in 1:length(KNN_result)
    println(result[i])
end

println("\nActual Species of Test Point:")
println(iris.Species[146])

KNN Algorithm Results:
((6.7, 3.0, 5.0), CategoricalValue{String,UInt8} "versicolor", 0.20000000000000018)
((6.9, 3.1, 5.4), CategoricalValue{String,UInt8} "virginica", 0.30000000000000027)
((6.5, 3.2, 5.1), CategoricalValue{String,UInt8} "virginica", 0.30000000000000043)
((6.8, 3.0, 5.5), CategoricalValue{String,UInt8} "virginica", 0.31622776601683766)

Actual Species of Test Point:
virginica


---
Of the 4 closest data points, one of them is a *versicolor* iris, while three are *virginica* irises, therefore the test point is most likely a *virginica* iris. Once we check the actual label of our test point, we see that it is in fact a *virginica* iris and that our algorithm predicted it correctly. We can further automate this algorithm by creating a function that counts the labels of the neighbors and simply returns the label of the majority.

---

In [60]:
# Function to interpret KNN algorithm results
function KNN_prediction(KNN_result)
    setosa_count = 0
    versicolor_count = 0
    virginica_count = 0
    
    for i in 1:length(KNN_result)
        if (KNN_result[i][2] == "setosa")
            setosa_count += 1
        elseif (KNN_result[i][2] == "versicolor")
            versicolor_count += 1
        elseif (KNN_result[i][2] == "virginica")
            virginica_count += 1
        end
    end
    
    # Determine greatest count
    if (virginica_count>versicolor_count && virginica_count>setosa_count)
        return "virginica"
    elseif (setosa_count>virginica_count && setosa_count>versicolor_count)
        return "setosa"
    elseif (versicolor_count>virginica_count && versicolor_count>setosa_count)
        return "versicolor"
    else
        return "inconclusive" # used when there is a tie for greatest count
    end

end


KNN_prediction (generic function with 1 method)

In [64]:
println("K-Nearest Neighbors:")
KNN_result = KNN(test_data[21], training_data, label_data, 4)
for i in 1:length(KNN_result)
    println(KNN_result[i])
end

println("\nPrediction:")
println(KNN_prediction(KNN_result))

K-Nearest Neighbors:
((6.8, 3.0, 5.5), CategoricalValue{String,UInt8} "virginica", 0.17320508075688737)
((6.7, 3.3, 5.7), CategoricalValue{String,UInt8} "virginica", 0.22360679774997896)
((6.5, 3.0, 5.5), CategoricalValue{String,UInt8} "virginica", 0.24494897427831783)
((6.9, 3.2, 5.7), CategoricalValue{String,UInt8} "virginica", 0.24494897427831822)

Prediction:
virginica


---
While this automation can make predictions on a larger scale easier as it removes the need more manual interpretation of results, there are certainly applications of the K-Nearest Neighbors algorithm that this type of automation is not appropriate for. One example of such an application where you would want to see all of the results with no automated interpretations would be an algorithm that recommends movies based on movies that you like. Here's an example of this type of application using some sample data from a listing of movies.

---

In [66]:
# Read in movie data
movie_data = CSV.read("movies_recommendation_data.csv")
# Run KNN algorithm and output results

,MovieID,MovieName,IMDBRating,Biography,Drama,Thriller
,Int64,String,Float64,Int64,Int64,Int64
1,58,The Imitation Game,8.0,1,1,1
2,8,Ex Machina,7.7,0,1,0
3,46,A Beautiful Mind,8.2,1,1,0
4,62,Good Will Hunting,8.3,0,1,0
5,97,Forrest Gump,8.8,0,1,0
6,98,21,6.8,0,1,0
7,31,Gifted,7.6,0,1,0
8,3,Travelling Salesman,5.9,0,1,0
9,51,Avatar,7.9,0,0,0


---
For this data, the variables we will be using for the x_data are the genre columns along with the IMDB rating, and the label data will be the movie names. When this data is run through the K-Nearest Neighbors algorithm, it will return the *k* nearest movies to it. In other words, the algorithm will take the movie that you feed in and return the movies that are most similar and you are most likely to enjoy.

---